# Google Gemini 2.0 with MCP (Model Context Protocol) Servers

Gemini models can be used with MCP server using its native tool calling capabilities. MCP, or Model Context Protocol, is an open standard introduced by Anthropic designed to standardize how AI models like Gemini interact with external tools and data sources. Instead of requiring custom integrations for each tool, MCP provides a structured way for models to access context, such as functions (tools), data sources (resources), or pre-defined prompts. This allows AI agents to securely and efficiently connect with real-world systems and workflows.

MCP server expose their tools via JSON schema definitions, which can be converted to Gemini compatible OpenAPI schema definitions. This allows you to easily use MCP server with Gemini models, below you will example on how to implement this.

You can learn more about Google Search integration with Gemini here:
- [https://ai.google.dev/gemini-api/docs/function-calling?lang=python](https://ai.google.dev/gemini-api/docs/function-calling?lang=python&example=weather)

In [1]:
# install Google GenAI and MCP
%pip install google-genai mcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


## Simple Example on how to use MCP with Gemini's tool calling

MCPs can be used with Google DeepMind Gemini by converting the MCP tools into Gemini compatible tools.


In [ ]:
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os

client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY")
)  # Replace with your actual API key setup


# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="npx",  # Executable
    args=[
        "-y",
        "@openbnb/mcp-server-airbnb",
        "--ignore-robots-txt",
    ],  # Optional command line arguments
    env=None,  # Optional environment variables
)

async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            prompt = "I want to book an apartment in Paris for 2 nights. 03/28 - 03/30"
            # Initialize the connection
            await session.initialize()

            # Get tools from MCP session and convert to Gemini Tool objects
            mcp_tools = await session.list_tools()
            tools = types.Tool(function_declarations=[
                {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.inputSchema,
                }
                for tool in mcp_tools.tools
            ])

            # Send request with function declarations
            response = client.models.generate_content(
                model="gemini-2.0-flash",  # Or your preferred model supporting function calling
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0.7,
                    tools=[tools],
                ),  # Example other config
            )
        # Check for a function call
        if response.candidates[0].content.parts[0].function_call:
            function_call = response.candidates[0].content.parts[0].function_call
            print(f"Function to call: {function_call.name}")
            print(f"Arguments: {function_call.args}")
            # In a real app, you would call your function here:
            # result = await session.call_tool(function_call.args, arguments=function_call.args)
            # sent new request with function call
        else:
            print("No function call found in the response.")
            print(response.text)

await run()


Function to call: airbnb_search
Arguments: {'checkout': '2024-03-30', 'checkin': '2024-03-28', 'location': 'Paris'}


## Full Agentic example with Gemini and Airbnb MCP

In [ ]:
from typing import List
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
model = "gemini-2.0-flash"

# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="npx",  # Executable
    args=[
        "-y",
        "@openbnb/mcp-server-airbnb",
        "--ignore-robots-txt",
    ],  # Optional command line arguments
    env=None,  # Optional environment variables
)

async def agent_loop(prompt: str, client: genai.Client, session: ClientSession):
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Initialize the connection
    await session.initialize()

    # --- 1. Get Tools from Session and convert to Gemini Tool objects ---
    mcp_tools = await session.list_tools()
    tools = types.Tool(function_declarations=[
        {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema,
        }
        for tool in mcp_tools.tools
    ])

    # --- 2. Initial Request with user prompt and function declarations ---
    response = await client.aio.models.generate_content(
        model=model,  # Or your preferred model supporting function calling
        contents=contents,
        config=types.GenerateContentConfig(
            temperature=0,
            tools=[tools],
        ),  # Example other config
    )

    # --- 3. Append initial response to contents ---
    contents.append(response.candidates[0].content)

    # --- 4. Tool Calling Loop ---
    turn_count = 0
    max_tool_turns = 5
    while response.function_calls and turn_count < max_tool_turns:
        turn_count += 1
        tool_response_parts: List[types.Part] = []

        # --- 4.1 Process all function calls in order and return in this turn ---
        for fc_part in response.function_calls:
            tool_name = fc_part.name
            args = fc_part.args or {}  # Ensure args is a dict
            print(f"Attempting to call MCP tool: '{tool_name}' with args: {args}")

            tool_response: dict
            try:
                # Call the session's tool executor
                tool_result = await session.call_tool(tool_name, args)
                print(f"MCP tool '{tool_name}' executed successfully.")
                if tool_result.isError:
                    tool_response = {"error": tool_result.content[0].text}
                else:
                    tool_response = {"result": tool_result.content[0].text}
            except Exception as e:
                tool_response = {"error":  f"Tool execution failed: {type(e).__name__}: {e}"}

            # Prepare FunctionResponse Part
            tool_response_parts.append(
                types.Part.from_function_response(
                    name=tool_name, response=tool_response
                )
            )

        # --- 4.2 Add the tool response(s) to history ---
        contents.append(types.Content(role="user", parts=tool_response_parts))
        print(f"Added {len(tool_response_parts)} tool response parts to history.")

        # --- 4.3 Make the next call to the model with updated history ---
        print("Making subsequent API call with tool responses...")
        response = await client.aio.models.generate_content(
            model=model,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[tools],
            ),  # Keep sending same config
        )
        contents.append(response.candidates[0].content)

    if turn_count >= max_tool_turns and response.function_calls:
        print(f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop.")

    print("MCP tool calling loop finished. Returning final response.")
    # --- 5. Return Final Response ---
    return response

async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "I want to book an apartment in Paris for 2 nights. 03/28 - 03/30"
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await agent_loop(prompt, client, session)
            return res
res = await run()
print(res.text)


Running agent loop with prompt: I want to book an apartment in Paris for 2 nights. 03/28 - 03/30
Attempting to call MCP tool: 'airbnb_search' with args: {'checkout': '2024-03-30', 'checkin': '2024-03-28', 'location': 'Paris'}
MCP tool 'airbnb_search' executed successfully.
Added 1 tool response parts to history.
Making subsequent API call with tool responses...
MCP tool calling loop finished. Returning final response.
OK. I have searched for apartments in Paris for 2 nights from 2024-03-28 to 2024-03-30. Here are some of the results:
* Room in Paris: https://www.airbnb.com/rooms/1192814572435529819
* Room with balcony very close to Paris: https://www.airbnb.com/rooms/1073307779802809893
* Small charming two-room in the middle of the courtyard: https://www.airbnb.com/rooms/51646829
* A room 30 minutes from Paris with the RER C. tram9: https://www.airbnb.com/rooms/16219327
* Le Voyage Paris: https://www.airbnb.com/rooms/1236913077555360681
* Small Functional Studio: https://www.airbnb.co